In [ ]:
import os
import matplotlib.pylab as plt
from glob import glob
import numpy as np

In [ ]:
PATH = os.path.abspath('data')

SOURCE_IMAGES = [os.path.join(PATH, "images/ds1"), os.path.join(PATH, "images/ds1/albumentated")]

images = []
labels = []

for si in SOURCE_IMAGES:
    images.extend(glob(os.path.join(si, "*.jpg")))
    labels.extend(glob(os.path.join(si.replace("images","labels"), "*.png")))

In [ ]:
images.sort()
labels.sort()

In [ ]:
print(np.size(images))
print(np.size(labels))

In [ ]:
x = [] # images
y = [] # labels

for img in images:
    full_size_image = plt.imread(img)
    x.append(full_size_image)

for lbl in labels:
    full_size_label = plt.imread(lbl,0)
    y.append(full_size_label)
    
del(images, labels, full_size_image, full_size_label)

In [ ]:
x = np.asarray(x)
y = np.asarray(y)

In [ ]:
print(y.min())
print(y.max())
print(x.shape)
print(y.shape)

In [ ]:
# Crop squared image
n_images, h, w = x.shape[0:3]

x = x[:,:,(w-h)//2:(w-(w-h)//2),:]
y = y[:,:,(w-h)//2:(w-(w-h)//2)]

n_images, h, w = x.shape[0:3]

print(x.shape)
print(y.shape)

In [ ]:
h_t, w_t = (512,512)
split_factor = 4 # We crop this number of smaller images out of one
x_t = np.zeros([n_images*split_factor, h_t, w_t, 3], dtype='uint8')
y_t = np.zeros([n_images*split_factor, h_t, w_t], dtype='uint8')

for i in range(n_images):
    x_t[i] = x[i,:h//2,:w//2,:]
    x_t[n_images+i] = x[i,:h//2,w//2:w,:]
    x_t[n_images*2+i] = x[i,h//2:h,:w//2,:]
    x_t[n_images*3+i] = x[i,h//2:h,w//2:w,:]
    y_t[i] = y[i,:h//2,:w//2]
    y_t[n_images+i] = y[i,:h//2,w//2:w]
    y_t[n_images*2+i] = y[i,h//2:h,:w//2]
    y_t[n_images*3+i] = y[i,h//2:h,w//2:w]
    
x = x_t
y = y_t
del(x_t,y_t)

In [ ]:
cs = ['background', 'asphalt', 'building', 'forest', 'grass', 'ground', 'roadAsphalt', 'roadGround', 'water']

In [ ]:
n_classes = len(cs)

from keras.utils import to_categorical

y = to_categorical(y, num_classes=n_classes)
y = y.reshape(x.shape[:3] + (n_classes,))
y.shape

In [ ]:
y = y.astype('int8')

In [ ]:
x = np.float32(x/255.)

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2
print("Train/test split: {}/{}\n".format(1-test_size,test_size))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)

del(x,y,x_train,y_train)

In [ ]:
print("X_test shape: {}".format(x_test.shape))
print("Y_test shape: {}\n".format(y_test.shape))

In [ ]:
from models.Unet import unet
from keras import optimizers

model = unet(input_size = x_test.shape[1:], n_classes=n_classes)

learning_rate = 5e-5
optimizer = optimizers.Adam(lr = learning_rate)
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

model.load_weights('weights/2018-10-09 20:19:40.hdf5')

In [ ]:
# a = model.evaluate(x=x[:2], y=y[:2], batch_size=1, verbose=1)

In [ ]:
batch_size = 1

y_pred = model.predict(x_test, batch_size=batch_size, verbose=1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator()
validation_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)

In [ ]:
import webcolors

hex_colors = ['#000000','#4A4A4A','#FF0000', '#407700', '#00FF37', '#8B572A', '#0000FF', '#FF7600', '#50E3C2']

rgb_colors = []

for hex_color in hex_colors:
    rgb_colors.append(webcolors.hex_to_rgb(hex_color))
    
def colorize(mask, colors):
    colorMask = np.zeros([mask.shape[0], mask.shape[1], 3],dtype=np.uint8)
    for r in range(mask.shape[0]):
        for c in range(mask.shape[1]):
            colorMask[r,c,] = colors[mask[r,c]]

    return colorMask

In [ ]:
n = 0

fig, axes = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(15,5)
axes[0].imshow(x_test[n])
axes[1].imshow(colorize(np.argmax(y_test[n],axis=-1), colors = np.array(rgb_colors)))
axes[2].imshow(colorize(np.argmax(y_pred[n],axis=-1), colors = np.array(rgb_colors)))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(15,5)
for i in range(len(x_test)):
    axes[0].set_title('RGB Image')
    axes[1].set_title('GT Mask')
    axes[2].set_title('Prediction Mask')
    axes[0].imshow(x_test[i])
    axes[1].imshow(colorize(np.argmax(y_test[i],axis=-1), colors = np.array(rgb_colors)))
    axes[2].imshow(colorize(np.argmax(y_pred[i],axis=-1), colors = np.array(rgb_colors)))
    fig.savefig('results/final/img{}.png'.format(i))